# Brazil - No Show Appointment - Deu Migué

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

This dataset collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment. A number of characteristics about the patient are included in each row.

You can download the dataset [here](https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd2e9a_noshowappointments-kagglev2-may-2016/noshowappointments-kagglev2-may-2016.csv).

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
df = pd.read_csv('brasil_migue.csv')

In [5]:
df.head(4)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No


## Columns descriptions and insights

The columns that is not listed below are considered self explained.
- **Scholarship** - Indicates whether or not the patient is enrolled in Brasilian welfare program Bolsa Família. 
    - Could indicate the real scholarship of the patient. The information if the patition is in the welfare could be another column.
    
- **ScheduledDay** - The day that the patient set up their appointment. 
    - In the first lines listed above is possible to see some problems in this data, some dates are the same or after the appointment day. The same day is a problem because in brazilian public hospiltals an appointment is scheduled only months before. This data does not seem truthful.

- **Neighbourhood** - The place where the hospital is located and not where the patient lives.
    - A data set with informations about the region could be useful. Only the the name of the place, where the hospital is located, become diffcult to link the place with the reason that the patient does not show up. The location where the patient live could be useful because some hospitals receive patients from another cities.

- **Handcap** - I am considering that the handicap means people with disabilities.

- **No-show** - Yes if the patient does not show up.
    - I could invert the data to mean that the 'yes' may be when the patient showed up. The columns name could be showed up in this case. This could avoid misunderstood.
    
- **SMS_received** - If the patient received a sms remembering about the appointment.
    - I have doubts about if this data is relevant. There is some reasons about my doubts that I am going to try to explain below:
        - This system of reminder is not common in Brazil
        - Some patients are enrolled in the welfare program Bolsa Família. Most certain that they do not have cellphones. This can be cheked more later in the data analysis.
        - The data do not explain when the sms was received.

> **Note**: The insights above are only my first impressions about the data. I am goint to do a more profund analise later in this document.  

<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

Display all the info about the data

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


There is no mising values in the data frame. Let's check if has duplicates values

In [8]:
sum(df.duplicated())

0

### Check unique values

In [9]:
df.Gender.unique()

array(['F', 'M'], dtype=object)

In [10]:
df.Scholarship.unique()

array([0, 1], dtype=int64)

In [11]:
df.Hipertension.unique()

array([1, 0], dtype=int64)

In [12]:
df.Diabetes.unique()

array([0, 1], dtype=int64)

In [13]:
df.Alcoholism.unique()

array([0, 1], dtype=int64)

In [14]:
df.Handcap.unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [15]:
df.SMS_received.unique()

array([0, 1], dtype=int64)

In [17]:
df['No-show'].unique()

array(['No', 'Yes'], dtype=object)

According with the data set documentation, the Handcap column should be true or false. Futhermore, I am going to check if this data has some correlation with the patients that do not show up.

### Check the date fields

First, let's convert to datetime the scheduled day and appointment day column

#### Convert to datetime

In [43]:
df.ScheduledDay = pd.to_datetime(df.ScheduledDay)
df.AppointmentDay = pd.to_datetime(df.AppointmentDay)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null datetime64[ns]
AppointmentDay    110527 non-null datetime64[ns]
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: datetime64[ns](2), float64(1), int64(8), object(3)
memory usage: 11.8+ MB


Now, I want to check how many Scheduled days are after the AppointmentDay

#### Check if ScheduledDay > AppointmentDay

In [69]:
df.query('ScheduledDay > AppointmentDay')

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29 18:38:08,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29 16:08:27,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29 16:19:04,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29 17:29:31,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29 16:07:23,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,No
8,5.639473e+13,5638447,F,2016-04-29 08:02:16,2016-04-29,21,ANDORINHAS,0,0,0,0,0,0,No
19,7.223289e+12,5640433,F,2016-04-29 10:43:14,2016-04-29,46,DA PENHA,0,0,0,0,0,0,No
24,8.734858e+12,5641780,F,2016-04-29 14:19:19,2016-04-29,65,TABUAZEIRO,0,0,0,0,0,0,No
26,2.578785e+10,5641781,F,2016-04-29 14:19:42,2016-04-29,45,BENTO FERREIRA,0,1,0,0,0,0,No
28,5.926172e+12,5642400,M,2016-04-29 15:48:02,2016-04-29,51,SÃO PEDRO,0,0,0,0,0,0,No


In [46]:
sum(df.ScheduledDay > df.AppointmentDay)

38568

In [49]:
sum(df.ScheduledDay == df.AppointmentDay)

0

I want to check if some patient in the results above received a sms

In [56]:
df.query('ScheduledDay > AppointmentDay and SMS_received == 1').SMS_received.count()

0

I am going to check if Appointment Day column has some value with time

In [61]:
sum(df.AppointmentDay.apply(lambda x: x.hour + x.minute + x.second + x.microsecond))

0

The column Appointment Day does not have time. I am going to check if has scheduled day value after the appointment day

In [110]:
different_day_array = []
for index, row in df.query('ScheduledDay > AppointmentDay').iterrows():
    if row['ScheduledDay'].day != row['AppointmentDay'].day:
        different_day_array.append(index)

In [111]:
df.loc[different_day_array].head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
27033,7.839273e+12,5679978,M,2016-05-10 10:51:53,2016-05-09,38,RESISTÊNCIA,0,0,0,0,1,0,Yes
55226,7.896294e+12,5715660,F,2016-05-18 14:50:41,2016-05-17,19,SANTO ANTÔNIO,0,0,0,0,1,0,Yes
64175,2.425226e+13,5664962,F,2016-05-05 13:43:58,2016-05-04,22,CONSOLAÇÃO,0,0,0,0,0,0,Yes
71533,9.982316e+14,5686628,F,2016-05-11 13:49:20,2016-05-05,81,SANTO ANTÔNIO,0,0,0,0,0,0,Yes
72362,3.787482e+12,5655637,M,2016-05-04 06:50:57,2016-05-03,7,TABUAZEIRO,0,0,0,0,0,0,Yes


The dataset has values with scheduled day after the appointment day. I am going to remove these data.
The appointment values does not have time, so the appointment date in the same day of scheduled date could be after the schedule. But in Brazil there is no possible that the scheduled and appointment day be the same, so I am going to remove the rows that contains schedule equal or after the appointment.

#### Clean ScheduledDay >= AppointmentDay

In [113]:
df = df.query('ScheduledDay < AppointmentDay')

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71959 entries, 5 to 110526
Data columns (total 14 columns):
PatientId         71959 non-null float64
AppointmentID     71959 non-null int64
Gender            71959 non-null object
ScheduledDay      71959 non-null datetime64[ns]
AppointmentDay    71959 non-null datetime64[ns]
Age               71959 non-null int64
Neighbourhood     71959 non-null object
Scholarship       71959 non-null int64
Hipertension      71959 non-null int64
Diabetes          71959 non-null int64
Alcoholism        71959 non-null int64
Handcap           71959 non-null int64
SMS_received      71959 non-null int64
No-show           71959 non-null object
dtypes: datetime64[ns](2), float64(1), int64(8), object(3)
memory usage: 8.2+ MB


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!